In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow import keras

# This code was tested with TensorFlow v1.4
print("You have TensorFlow version", tf.__version__)

/home/chetan/.local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


You have TensorFlow version 1.7.0


In [3]:
import pandas as pd
import numpy as np
file_csv = pd.read_csv('export.csv')
data = file_csv[["DESCR", "PRODUCT"]]
data = data.drop_duplicates(subset=["DESCR", "PRODUCT"], keep=False)

/home/chetan/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,5,6,9,13,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
numpy_array = data.as_matrix()
X = numpy_array[:,0]
Y = numpy_array[:,1]
from sklearn.model_selection import train_test_split
train_DESCR, test_DESCR, train_PRODUCT, test_PRODUCT = train_test_split(X, Y, test_size=0.2, random_state=8427529)

In [5]:
max_words = 12000
tokenize = keras.preprocessing.text.Tokenizer(num_words=max_words, char_level=False)

In [6]:
tokenize.fit_on_texts(train_DESCR) # only fit on train
x_train = tokenize.texts_to_matrix(train_DESCR).astype(np.float32) # model expects float32
x_test = tokenize.texts_to_matrix(test_DESCR).astype(np.float32)

In [7]:
# Use sklearn utility to convert label strings to numbered index
encoder = LabelEncoder()
encoder.fit(train_PRODUCT)
y_train = encoder.transform(train_PRODUCT)
y_test = encoder.transform(test_PRODUCT)

In [8]:
# Converts the labels to a one-hot representation
num_classes = np.max(y_train) + 1
y_train = keras.utils.to_categorical(y_train, num_classes).astype(np.float32)
y_test = keras.utils.to_categorical(y_test, num_classes).astype(np.float32)

In [9]:
# Inspect the dimenstions of our training and test data (this is helpful to debug)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (73680, 12000)
x_test shape: (18420, 12000)
y_train shape: (73680, 119)
y_test shape: (18420, 119)


In [10]:
# This model trains very quickly and 2 epochs are already more than enough
# Training for more epochs will likely lead to overfitting on this dataset
# You can try tweaking these hyperparamaters when using this model with your own data
batch_size = 24
epochs = 3

In [11]:
# Build the model
model = keras.models.Sequential()
model.add(keras.layers.Dense(512, input_shape=(max_words,), name="descr"))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(num_classes))
model.add(keras.layers.Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [12]:
# Convert Keras model to estimator
estimator_model = keras.estimator.model_to_estimator(keras_model=model)

INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f88541aefd0>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/tmp/tmpbDZQlH', '_save_summary_steps': 100}


In [15]:
# Define input fn
def input_function(features, labels=None, shuffle=False):
    input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"descr_input": features}, # See the accompanying blog post for explanation on naming
        y=labels,
        shuffle=shuffle
    )
    return input_fn


In [16]:
estimator_model.train(input_fn=input_function(x_train, y_train, shuffle=True))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpbDZQlH/keras_model.ckpt
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpbDZQlH/model.ckpt.
INFO:tensorflow:loss = 4.779981, step = 1
INFO:tensorflow:global_step/sec: 8.62529
INFO:tensorflow:loss = 1.0863581, step = 101 (11.596 sec)
INFO:tensorflow:global_step/sec: 8.99741
INFO:tensorflow:loss = 0.35556793, step = 201 (11.113 sec)
INFO:tensorflow:global_step/sec: 8.97527
INFO:tensorflow:loss = 0.3160286, step = 301 (11.142 sec)
INFO:tensorflow:global_step/sec: 8.67637
INFO:tensorflow:loss = 0.2592278, step = 401 (11.525 sec)
INFO:tensorflow:global_step/sec: 8.77858
INFO:tensorflow:loss = 0.18682808, step = 501 (11.391 sec)
INFO:tensorflow:Saving checkpoints for 577 into /tmp/tmpbDZQlH/model.ckpt.
IN

In [17]:
# Evaluate the accuracy of our trained model
score = estimator_model.evaluate(input_function(x_test, labels=y_test))
print(score)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-27-10:44:47
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpbDZQlH/model.ckpt-577
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-27-10:44:51
INFO:tensorflow:Saving dict for global step 577: accuracy = 0.97045803, global_step = 577, loss = 0.13203448
{'loss': 0.13203448, 'global_step': 577, 'accuracy': 0.97045803}


In [18]:
test_PRODUCT = pd.DataFrame(data = test_PRODUCT, columns=["PRODUCT"])

In [19]:
# Here's how to generate a prediction on individual examples
text_labels = encoder.classes_ 

# We'll make predictions for the first five examples
examples = x_test[:10]
predictions = list(estimator_model.predict(input_function(examples)))

# Print out the true and expected label
for i in range(len(examples)):
    prediction_array = list(predictions[i].values())[0]
    predicted_label = text_labels[np.argmax(prediction_array)]
    print('Actual label: ' + ''.join(test_PRODUCT.values[i]))
    print("Predicted label: " + predicted_label + "\n")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpbDZQlH/model.ckpt-577
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Actual label: Clavamox Drops
Predicted label: Clavamox Drops

Actual label: Baytril Tablet
Predicted label: Baytril Tablet

Actual label: Prednisone Tablet
Predicted label: Prednisone Tablet

Actual label: Prednisone Tablet
Predicted label: Prednisone Tablet

Actual label: Prednisolone Suspension
Predicted label: Prednisolone Suspension

Actual label: Metronidazole Unspecified
Predicted label: Metronidazole Unspecified

Actual label: Otitis Compound Pharmacy
Predicted label: Otitis Compound Pharmacy

Actual label: Clavamox Chewable/Tablet Unspecified
Predicted label: Clavamox Chewable/Tablet Unspecified

Actual label: Baytril Injectable
Predicted label: Baytril Injectable

Actual label: Clavamox Chewable/Tablet Unspecified
Pr